In [2]:
import pandas as pd
import pandahouse as ph
import matplotlib.pyplot as plt
import numpy as np
import hashlib
from scipy import stats
import seaborn as sns

In [3]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20221020'
}

q1 = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2022-10-03' and '2022-10-09'
    and exp_group in (0,3)
GROUP BY exp_group, user_id
"""
q2 = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2022-10-03' and '2022-10-09'
    and exp_group in (2,1)
GROUP BY exp_group, user_id
"""

df1 = ph.read_clickhouse(q1, connection=connection)
df2 = ph.read_clickhouse(q2, connection=connection)


## t-тест для групп 0 и 3 ##

In [4]:
#Проведем t-тест на исходных данных для групп 0 и 3
# t-критерий Стьюдента
stats.ttest_ind(df1[df1.exp_group == 0].ctr,
                df1[df1.exp_group == 3].ctr,
                equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [5]:
CTR_control = (df1[df1.exp_group == 0].likes.sum())/(df1[df1.exp_group == 0].views.sum())


In [6]:
pd.options.mode.chained_assignment = None

In [7]:
df1['linearized_likes'] = ''
for i in range(len(df1)):
    df1['linearized_likes'][i] = df1['likes'][i] - CTR_control*df1['views'][i]


In [8]:
# t-критерий Стьюдента для линеаризованных лайков
stats.ttest_ind(df1[df1.exp_group == 0].linearized_likes,
                df1[df1.exp_group == 3].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=-15.214995460903783, pvalue=5.491424947972764e-52)

### Отличия как были статистически значимыми, так и остались. Значение p-value уменьшилось. ###

## t-тест для групп 1 и 2 ##

In [9]:
#Проведем t-тест на исходных данных для групп 2 и 1
# t-критерий Стьюдента
stats.ttest_ind(df2[df2.exp_group == 2].ctr,
                df2[df2.exp_group == 1].ctr,
                equal_var=False)

Ttest_indResult(statistic=-0.7094392041270486, pvalue=0.4780623130874935)

In [10]:
CTR_control_1 = (df2[df2.exp_group == 1].likes.sum())/(df2[df2.exp_group == 1].views.sum())

In [11]:
df2['linearized_likes'] = ''
for i in range(len(df2)):
    df2['linearized_likes'][i] = df2['likes'][i] - CTR_control_1*df2['views'][i]

In [14]:
# t-критерий Стьюдента для линеаризованных лайков
stats.ttest_ind(df2[df2.exp_group == 2].linearized_likes,
                df2[df2.exp_group == 1].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=-6.122579994775915, pvalue=9.43943218704115e-10)

### Для этих же групп способ с линеаризованными лайками в корни изменил ситуацию. Значение p-value уменьшилось и результаты стали статистичсеки значимыми. ###